In [1]:
from rdkit import Chem
from copy import deepcopy
from src.config import filepaths
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from IPython.display import SVG
import json
import ipywidgets as widgets
from ipywidgets import interact
from src.chem_draw import draw_molecule

In [2]:
with open(filepaths['sprhea'] /  "sprhea_240310_v3_mapped_no_subunits.json", 'r') as f:
    rhea = json.load(f)

In [3]:
smiles = {}
for k, v in rhea.items():
    rxn_smiles = v['smarts']
    lhs, rhs = [side.split(".") for side in rxn_smiles.split(">>")]
    row_smiles = lhs + rhs
    for smi in row_smiles:
        smiles[smi] = k

print(len(smiles))

8603


In [4]:
qty_C = '[#6]-[#6](-[#6])(-[#6])(-[#6])'
qty_C_smiles = {}
for k, v in smiles.items():
    mol = Chem.MolFromSmiles(k)
    patt = Chem.MolFromSmarts(qty_C)

    if mol.HasSubstructMatch(patt):
        qty_C_smiles[k] = v

print(len(qty_C_smiles))

[16:03:37] Unusual charge on atom 0 number of radical electrons set to zero
[16:03:37] WARNING: not removing hydrogen atom with dummy atom neighbors
[16:03:37] WARNING: not removing hydrogen atom with dummy atom neighbors
[16:03:37] WARNING: not removing hydrogen atom with dummy atom neighbors
[16:03:38] Unusual charge on atom 0 number of radical electrons set to zero


1828


In [5]:
# Create a dropdown widget with the keys of qty_C_smiles
dropdown = widgets.Dropdown(
    options=sorted(qty_C_smiles.keys(), key=lambda x: Chem.MolFromSmiles(x).GetNumAtoms()),
    description='SMILES:',
    style={'description_width': 'initial'}
)

# Define a function to display the selected key
@interact
def display_molecule(smi = dropdown):
    display(SVG(draw_molecule(smi, size=(300, 300))))

interactive(children=(Dropdown(description='SMILES:', options=('CC(C)(C)C=O', 'CC(=O)C(C)(C)C', 'CC(C)(C)C(O)C…